### 16.3 Chef Crew

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [3]:
from crewai import Crew, Agent, Task

head_chef = Agent(
    role="Head Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home.",
    backstory=""" 
    You are a famous chef that specializes in cuisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
)

vegan_chef = Agent(
    role="Vegan Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory=""" 
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

In [4]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=head_chef,
    expected_output=""" 
    Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions"
    """,
    output_file="normal_recipe.md",
)

vegan_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=vegan_chef,
    expected_output=""" 
    Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.
    """,
    output_file="vegan_recipe.md",
)

In [6]:
crew = Crew(
    tasks=[
        normal_recipe,
        vegan_recipe
    ],
    agents=[
        head_chef,
        vegan_chef
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs={
        "dish": "Greek dinner",
        "people": "5",
    }
)

 
[2025-03-11 22:14:47][🚀 CREW 'CREW' STARTED, 63F04106-E598-49F0-AFAC-98E52CB229C1]: 2025-03-11 22:14:47.567471
 
[2025-03-11 22:14:47][📋 TASK STARTED: COME UP WITH A GREEK DINNER THAT SERVES 5 PEOPLE.]: 2025-03-11 22:14:47.583484
 
[2025-03-11 22:14:47][🤖 AGENT 'HEAD CHEF' STARTED TASK]: 2025-03-11 22:14:47.585485
# Agent: Head Chef
## Task: Come up with a Greek dinner that serves 5 people.
 
[2025-03-11 22:14:47][🤖 LLM CALL STARTED]: 2025-03-11 22:14:47.586484
 
[2025-03-11 22:14:59][✅ LLM CALL COMPLETED]: 2025-03-11 22:14:59.595346


# Agent: Head Chef
## Final Answer: 
**Ingredients Required:**

*For the main dish (Greek Moussaka):*  
- 3 medium eggplants, sliced into 1/4 inch rounds  
- 1 pound ground beef or lamb  
- 1 large onion, diced  
- 2 cloves garlic, minced  
- 1 can (14 oz) crushed tomatoes  
- 1 tablespoon tomato paste  
- 1 teaspoon dried oregano  
- 1 teaspoon cinnamon  
- 1/2 teaspoon salt  
- 1/2 teaspoon black pepper  
- 3 medium potatoes, peeled and thinly sliced